In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()
# this is an object used to interact with spark, the entry-point

24/08/29 15:17:01 WARN Utils: Your hostname, RaulPC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/29 15:17:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/08/29 15:17:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2024-08-29 15:22:35--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.165.247.79, 3.165.247.187, 3.165.247.72, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.165.247.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet’

fhvhv_tripdata_2021 100%[===================>] 294.61M  8.57MB/s    in 39s     

2024-08-29 15:23:14 (7.65 MB/s) - ‘fhvhv_tripdata_2021-01.parquet’ saved [308924937/308924937]



In [4]:
df = spark.read \
    .option("header", "true") \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [7]:
df.head(5) # it does not infer the type of csv, but parquet does not have this problem

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 1, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 1, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 1, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 1, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 1, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 1, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 2, 18, 21), PULocationID=152, DO

In [9]:
df.schema   # everything is a string in csv (not here btw), check the code of ex 4 week 5 for the infering of types

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [14]:
# we want to create many partitions from the large single file
df = df.repartition(24)   # this is lazy, does not trigger until the next action is called

In [15]:
df.write.parquet('fhvhv/2021/01/', mode="overwrite")

24/08/29 15:38:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


24/08/29 15:38:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


24/08/29 15:39:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


24/08/29 15:39:03 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [17]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [19]:
df.select('pickup_datetime', 'dropoff_datetime').filter(df.hvfhs_license_num == 'HV0003').show()  # filter is lazy

+-------------------+-------------------+
|    pickup_datetime|   dropoff_datetime|
+-------------------+-------------------+
|2021-01-12 16:30:40|2021-01-12 17:08:15|
|2021-01-31 15:51:25|2021-01-31 16:14:06|
|2021-01-02 17:56:36|2021-01-02 18:43:44|
|2021-01-16 19:31:36|2021-01-16 19:42:09|
|2021-01-08 22:18:46|2021-01-08 22:25:25|
|2021-01-21 13:40:46|2021-01-21 13:48:52|
|2021-01-16 20:03:40|2021-01-16 20:10:20|
|2021-01-04 06:24:56|2021-01-04 06:30:51|
|2021-01-08 15:22:38|2021-01-08 15:34:42|
|2021-01-16 21:35:50|2021-01-16 21:47:50|
|2021-01-24 22:32:43|2021-01-24 22:57:19|
|2021-01-20 00:25:30|2021-01-20 00:41:13|
|2021-01-15 16:42:20|2021-01-15 16:52:34|
|2021-01-28 20:33:36|2021-01-28 20:49:19|
|2021-01-10 20:39:03|2021-01-10 20:46:45|
|2021-01-10 01:26:38|2021-01-10 01:34:48|
|2021-01-22 17:03:38|2021-01-22 17:20:57|
|2021-01-27 21:24:35|2021-01-27 21:39:28|
|2021-01-20 03:05:18|2021-01-20 03:26:38|
|2021-01-17 15:36:45|2021-01-17 15:42:07|
+-------------------+-------------

In [20]:
from pyspark.sql import functions as F

In [23]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

 # transformation that adds new column pickup_date and insert the to_date(pickup_datetime)

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-26|  2021-01-26|         247|         200|
| 2021-01-29|  2021-01-29|          61|         119|
| 2021-01-27|  2021-01-27|         248|         182|
| 2021-01-04|  2021-01-04|         225|          97|
| 2021-01-25|  2021-01-25|         233|         140|
| 2021-01-15|  2021-01-15|          20|         265|
| 2021-01-23|  2021-01-23|         137|         226|
| 2021-01-20|  2021-01-20|         126|         254|
| 2021-01-23|  2021-01-23|          50|         166|
| 2021-01-01|  2021-01-01|          28|         129|
| 2021-01-23|  2021-01-23|          68|         265|
| 2021-01-28|  2021-01-28|          28|         134|
| 2021-01-15|  2021-01-15|         159|          18|
| 2021-01-05|  2021-01-05|          37|         256|
| 2021-01-04|  2021-01-04|          35|         177|
| 2021-01-17|  2021-01-17|          85|       

In [30]:
from pyspark.sql import types

In [24]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [31]:
# you can create user defined functions (UDF) like this 
crazy_stuff_udf=F.udf(crazy_stuff,  returnType= types.StringType())
# and then you apply it similarly to the others

In [32]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num))\
    .select('pickup_date', 'dropoff_date', 'base_id', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+-------+------------+------------+
|pickup_date|dropoff_date|base_id|PULocationID|DOLocationID|
+-----------+------------+-------+------------+------------+
| 2021-01-19|  2021-01-19|  e/b3b|         208|         254|
| 2021-01-12|  2021-01-12|  s/b44|          72|          72|
| 2021-01-22|  2021-01-22|  e/b38|          72|          39|
| 2021-01-17|  2021-01-17|  e/9ce|         198|         196|
| 2021-01-06|  2021-01-06|  s/b3d|         169|          74|
| 2021-01-05|  2021-01-05|  e/b38|         165|         225|
| 2021-01-12|  2021-01-12|  e/b3e|          56|         129|
| 2021-01-05|  2021-01-05|  e/9ce|         225|          97|
| 2021-01-13|  2021-01-13|  s/b3d|          37|          37|
| 2021-01-23|  2021-01-23|  e/b30|          87|         107|
| 2021-01-22|  2021-01-22|  e/9ce|         182|         127|
| 2021-01-16|  2021-01-16|  e/acc|          17|         188|
| 2021-01-09|  2021-01-09|  e/9ce|          61|         188|
| 2021-01-14|  2021-01-1